**Корректность проверена на Python 3.7:**
+ pandas 0.23.0
+ numpy 1.14.5
+ scipy 1.1.0
+ statsmodels 0.9.0

# Практика построения регрессии

In [1]:
import warnings
warnings.filterwarnings('ignore')

In [2]:
import statsmodels
import scipy as sc
import numpy as np
import pandas as pd
import statsmodels.formula.api as smf
import statsmodels.stats.api as sms
from statsmodels.graphics.regressionplots import plot_leverage_resid2
import matplotlib.pyplot as plt

In [3]:
print(np.__version__)
print(pd.__version__)
print(sc.__version__)
print(statsmodels.__version__)

1.16.2
0.24.2
1.2.1
0.9.0


In [4]:
%pylab inline

Populating the interactive namespace from numpy and matplotlib


## Постановка

Давайте проанализируем данные опроса 4361 женщин из Ботсваны:

botswana.tsv

О каждой из них мы знаем:

сколько детей она родила (признак ceb)
возраст (age)
длительность получения образования (educ)
религиозная принадлежность (religion)
идеальное, по её мнению, количество детей в семье (idlnchld)
была ли она когда-нибудь замужем (evermarr)
возраст первого замужества (agefm)
длительность получения образования мужем (heduc)
знает ли она о методах контрацепции (knowmeth)
использует ли она методы контрацепции (usemeth)
живёт ли она в городе (urban)
есть ли у неё электричество, радио, телевизор и велосипед (electric, radio, tv, bicycle)
Давайте научимся оценивать количество детей ceb по остальным признакам.

Загрузите данные и внимательно изучите их. Сколько разных значений принимает признак religion?

In [11]:
raw_data = pd.read_csv("botswana.tsv", sep="\t", index_col=False) 
raw_data.head(3)

,ceb,age,educ,religion,idlnchld,knowmeth,usemeth,evermarr,agefm,heduc,urban,electric,radio,tv,bicycle
0,0,18,10,catholic,4.0,1.0,1.0,0,NaN,NaN,1,1.0,1.0,1.0,1.0
1,2,43,11,protestant,2.0,1.0,1.0,1,20.0,14.0,1,1.0,1.0,1.0,1.0
2,0,49,4,spirit,4.0,1.0,0.0,1,22.0,1.0,1,1.0,1.0,0.0,0.0


In [12]:
raw_data.religion.describe().unique

<bound method Series.unique of count       4361
unique         4
top       spirit
freq        1841
Name: religion, dtype: object>

Во многих признаках есть пропущенные значения. Сколько объектов из 4361 останется, если выбросить все, содержащие пропуски?

In [13]:
count_ok = 0
count_no = 0
display(raw_data.shape)

for i in range(raw_data.shape[0]):
    if raw_data[raw_data.index.isin([i])].notnull().any().all() == True:
        count_ok += 1
    if raw_data[raw_data.index.isin([i])].isnull().any().any() == True:
        count_no += 1
print count_ok, count_no, count_ok+count_no

(4361, 15)

1834 2527 4361


В разных признаках пропуски возникают по разным причинам и должны обрабатываться по-разному.

Например, в признаке agefm пропуски стоят только там, где evermarr=0, то есть, они соответствуют женщинам, никогда не выходившим замуж. Таким образом, для этого признака NaN соответствует значению "не применимо".

В подобных случаях, когда признак 
x1​ на части объектов в принципе не может принимать никакие значения, рекомендуется поступать так:

создать новый бинарный признак 
  x​ 
= {
1, x​1='не применимо',
0,иначе;

заменить "не применимо" в x1​ на произвольную константу c, которая среди других значений x1​ не встречается.

Теперь, когда мы построим регрессию на оба признака и получим модель вида

y = beta0​ + beta1​ x1 + beta2​ x2,

на тех объектах, где x1 было измерено, регрессионное уравнение примет вид

y = beta0 + beta1​ x1,

а там, где x1 было "не применимо", получится

y=beta0 + beta1​ c + beta2.

Выбор c влияет только на значение и интерпретацию beta_{2} , но не beta1 .

Давайте используем этот метод для обработки пропусков в agefm и heduc.

Создайте признак nevermarr, равный единице там, где в agefm пропуски.

Удалите признак evermarr — в сумме с nevermarr он даёт константу, значит, в нашей матрице X будет мультиколлинеарность.

Замените NaN в признаке agefm на $\c_{agefm}=0

У объектов, где nevermarr = 1, замените NaN в признаке heduc на c_{heduc_1}=-1 
(ноль использовать нельзя, так как он уже встречается у некоторых объектов выборки).

Сколько осталось пропущенных значений в признаке heduc?

In [101]:
# Создайте признак nevermarr, равный единице там, где в agefm пропуски
data = raw_data.copy()
data['nevermarr'] = 0

In [104]:
display(data.head(5))
data.shape

,ceb,age,educ,religion,idlnchld,knowmeth,usemeth,evermarr,agefm,heduc,urban,electric,radio,tv,bicycle,nevermarr
0,0,18,10,catholic,4.0,1.0,1.0,0,NaN,NaN,1,1.0,1.0,1.0,1.0,0
1,2,43,11,protestant,2.0,1.0,1.0,1,20.0,14.0,1,1.0,1.0,1.0,1.0,0
2,0,49,4,spirit,4.0,1.0,0.0,1,22.0,1.0,1,1.0,1.0,0.0,0.0,0
3,0,24,12,other,2.0,1.0,0.0,0,NaN,NaN,1,1.0,1.0,1.0,1.0,0
4,3,32,13,other,3.0,1.0,1.0,1,24.0,12.0,1,1.0,1.0,1.0,1.0,0


(4361, 16)

In [105]:
data.loc[data.agefm.isnull(), 'nevermarr'] = 1

In [107]:
display(data.shape)
data.head(5)

(4361, 16)

,ceb,age,educ,religion,idlnchld,knowmeth,usemeth,evermarr,agefm,heduc,urban,electric,radio,tv,bicycle,nevermarr
0,0,18,10,catholic,4.0,1.0,1.0,0,NaN,NaN,1,1.0,1.0,1.0,1.0,1
1,2,43,11,protestant,2.0,1.0,1.0,1,20.0,14.0,1,1.0,1.0,1.0,1.0,0
2,0,49,4,spirit,4.0,1.0,0.0,1,22.0,1.0,1,1.0,1.0,0.0,0.0,0
3,0,24,12,other,2.0,1.0,0.0,0,NaN,NaN,1,1.0,1.0,1.0,1.0,1
4,3,32,13,other,3.0,1.0,1.0,1,24.0,12.0,1,1.0,1.0,1.0,1.0,0


In [108]:
# Удалите признак evermarr — в сумме с nevermarr он даёт константу, значит, 
# в нашей матрице X будет мультиколлинеарность.
data.drop(['evermarr'],  axis=1, inplace=True)
display(data.shape)
data.head()

(4361, 15)

,ceb,age,educ,religion,idlnchld,knowmeth,usemeth,agefm,heduc,urban,electric,radio,tv,bicycle,nevermarr
0,0,18,10,catholic,4.0,1.0,1.0,NaN,NaN,1,1.0,1.0,1.0,1.0,1
1,2,43,11,protestant,2.0,1.0,1.0,20.0,14.0,1,1.0,1.0,1.0,1.0,0
2,0,49,4,spirit,4.0,1.0,0.0,22.0,1.0,1,1.0,1.0,0.0,0.0,0
3,0,24,12,other,2.0,1.0,0.0,NaN,NaN,1,1.0,1.0,1.0,1.0,1
4,3,32,13,other,3.0,1.0,1.0,24.0,12.0,1,1.0,1.0,1.0,1.0,0


In [109]:
# Замените NaN в признаке agefm на c_{agefm}=0
data.loc[data.agefm.isnull(), 'agefm'] = 0
display(data.shape)
data.head()

(4361, 15)

,ceb,age,educ,religion,idlnchld,knowmeth,usemeth,agefm,heduc,urban,electric,radio,tv,bicycle,nevermarr
0,0,18,10,catholic,4.0,1.0,1.0,0.0,NaN,1,1.0,1.0,1.0,1.0,1
1,2,43,11,protestant,2.0,1.0,1.0,20.0,14.0,1,1.0,1.0,1.0,1.0,0
2,0,49,4,spirit,4.0,1.0,0.0,22.0,1.0,1,1.0,1.0,0.0,0.0,0
3,0,24,12,other,2.0,1.0,0.0,0.0,NaN,1,1.0,1.0,1.0,1.0,1
4,3,32,13,other,3.0,1.0,1.0,24.0,12.0,1,1.0,1.0,1.0,1.0,0


In [110]:
# У объектов, где nevermarr = 1, замените NaN в признаке heduc на c_{heduc_1}=-1 
# (ноль использовать нельзя, так как он уже встречается у некоторых объектов выборки).
display(data.loc[data.nevermarr == 1  ].shape)
data.loc[data.heduc.isnull() ].shape

(2282, 15)

(2405, 15)

In [111]:
cond1 = data.nevermarr == 1
cond2 = data.heduc.isnull()
data.loc[cond1 & cond2, 'heduc'] = -1

In [112]:
# Сколько осталось пропущенных значений в признаке heduc?
data[data.heduc.isnull() == True].shape

(123, 15)

Избавимся от оставшихся пропусков.

Для признаков idlnchld, heduc и usemeth проведите операцию, аналогичную предыдущей: создайте индикаторы пропусков по этим признакам (idlnchld_noans, heduc_noans, usemeth_noans), замените пропуски на нехарактерные значения 
(c idlnchld = −1, 
c heduc 2 = −2 (значение -1 мы уже использовали), 
c usemeth = −1).

Остались только пропуски в признаках knowmeth, electric, radio, tv и bicycle. Их очень мало, так что удалите объекты, на которых их значения пропущены.

Какого размера теперь наша матрица данных? Умножьте количество строк на количество всех столбцов (включая отклик ceb).

In [113]:
# создайте индикаторы пропусков по признакам idlnchld_noans, heduc_noans, usemeth_noans
data_1 = data.copy()
display(data_1.shape)

(4361, 15)

In [114]:
new_indicators = ['idlnchld_noans', 'heduc_noans', 'usemeth_noans']
for i in new_indicators:
    data_1[i] = 0
display(data_1.shape)

(4361, 18)

In [115]:
data_1.loc[data_1.idlnchld.isnull(), 'idlnchld_noans'] = 1
data_1.loc[data_1.heduc.isnull(), 'heduc_noans'] = 1
data_1.loc[data_1.usemeth.isnull(), 'usemeth_noans'] = 1

In [116]:
# замените пропуски на нехарактерные значения (c idlnchld = −1, c heduc 2 = −2 
# (значение -1 мы уже использовали), c usemeth = −1).
cond_1 = data_1.idlnchld_noans == 1
cond_2 = data_1.idlnchld.isnull()
data_1.loc[cond_1 & cond_2, 'idlnchld'] = -1

cond_3 = data_1.heduc_noans == 1
cond_4 = data_1.heduc.isnull()
data_1.loc[cond_3 & cond_4, 'heduc'] = -2

cond_5 = data_1.usemeth_noans == 1
cond_6 = data_1.usemeth.isnull()
data_1.loc[cond_5 & cond_6, 'usemeth'] = -1

display(data_1.shape)

(4361, 18)

In [152]:
# Остались только пропуски в признаках knowmeth, electric, radio, tv и bicycle. 
# Их очень мало, так что удалите объекты, на которых их значения пропущены.
data_2 = data_1.copy()
data_2.shape

(4361, 18)

In [153]:
drop_indexes = []
to_drop_nulls = ['knowmeth', 'electric', 'radio', 'tv', 'bicycle']
for j in to_drop_nulls:
    for i in data_2[data_2[j].isnull()].index:
        drop_indexes.append(i)
print drop_indexes

[2353, 3748, 4001, 4172, 4173, 4248, 4260, 821, 1179, 4243, 3931, 4243, 1179, 4243, 114, 1327, 4243]


In [154]:
new_to_drop = list(set(drop_indexes))
new_to_drop

[4001, 3931, 3748, 4172, 4173, 1327, 2353, 114, 4243, 821, 4248, 4260, 1179]

In [155]:
data_2.drop(index=new_to_drop, axis=0, inplace=True)
data_2.shape

(4348, 18)

In [158]:
data_1.shape[0] - len(new_to_drop)

4348

In [159]:
data_2.shape[0] * data_2.shape[1]

78264

Постройте регрессию количества детей ceb на все имеющиеся признаки методом smf.ols, как в разобранном до этого примере. Какой получился коэффициент детерминации R^2? 
Округлите до трёх знаков после десятичной точки.

In [160]:
data_2.columns

Index([u'ceb', u'age', u'educ', u'religion', u'idlnchld', u'knowmeth',
       u'usemeth', u'agefm', u'heduc', u'urban', u'electric', u'radio', u'tv',
       u'bicycle', u'nevermarr', u'idlnchld_noans', u'heduc_noans',
       u'usemeth_noans'],
      dtype='object')

In [175]:
#строим функцией ols statsmodels, ceb - отклик (целевой показатель)

m_2 = smf.ols('ceb ~ age + educ + religion + idlnchld + knowmeth + '\
       'usemeth + agefm + heduc + urban + electric + radio + tv + '\
       'bicycle + nevermarr + idlnchld_noans + heduc_noans + usemeth_noans', 
             data=data_2)
fitted_2 = m_2.fit()
print(fitted.summary())

                            OLS Regression Results                            
Dep. Variable:                    ceb   R-squared:                       0.644
Model:                            OLS   Adj. R-squared:                  0.643
Method:                 Least Squares   F-statistic:                     412.5
Date:                Thu, 21 Nov 2019   Prob (F-statistic):               0.00
Time:                        21:18:06   Log-Likelihood:                -7732.1
No. Observations:                4348   AIC:                         1.550e+04
Df Residuals:                    4328   BIC:                         1.563e+04
Df Model:                          19                                         
Covariance Type:            nonrobust                                         
                             coef    std err          t      P>|t|      [0.025      0.975]
------------------------------------------------------------------------------------------
Intercept                 -1

Обратите внимание, что для признака religion в модели автоматически создалось несколько бинарных фиктивных переменных. Сколько их?

3

Проверьте критерием Бройша-Пагана гомоскедастичность ошибки в построенной модели. Выполняется ли она?

Если ошибка гетероскедастична, перенастройте модель, сделав поправку Уайта типа HC1.

In [185]:
print('Breusch-Pagan test: p=%f' % sms.het_breuschpagan(fitted_2.resid, fitted_2.model.exog)[1])

Breusch-Pagan test: p=0.000000


In [206]:
m_2 = smf.ols('ceb ~ age + educ + religion + idlnchld + knowmeth + '\
       'usemeth + agefm + heduc + urban + electric + radio + tv + '\
       'bicycle + nevermarr + idlnchld_noans + heduc_noans + usemeth_noans', 
             data=data_2)
fitted_2_u = m_2.fit(cov_type='HC1')

Он нулевой. Гипотеза отвергается. Ошибки гетероскедастичны, значит, значимость признаков может определяться неверно.

Удалите из модели незначимые признаки religion, radio и tv. Проверьте гомоскедастичность ошибки, при необходимости сделайте поправку Уайта.

In [203]:
data_3 = data_2.copy()
data_3.drop(columns=['religion', 'radio', 'tv'], axis=1, inplace=True)
data_3.shape

(4348, 15)

In [204]:
m_3 = smf.ols('ceb ~ age + educ + idlnchld + knowmeth + '\
       'usemeth + agefm + heduc + urban + electric + '\
       'bicycle + nevermarr + idlnchld_noans + heduc_noans + usemeth_noans', 
             data=data_3)
fitted_3 = m_3.fit()

In [205]:
print('Breusch-Pagan test: p=%f' % sms.het_breuschpagan(fitted_3.resid, fitted_3.model.exog)[1])

Breusch-Pagan test: p=0.000000


Он нулевой. Гипотеза отвергается. Ошибки гетероскедастичны, значит, значимость признаков может определяться неверно.

Сделаем поправку Уайта. Она пересчитывает значения диспресий центров регрессии с учетом того, что в модели присутствует гетероскедаксичность. Этот метод не используют всегда, по тому, что он менее точен, когда гетероскедаксичности нет.

In [193]:
m_3 = smf.ols('ceb ~ age + educ + idlnchld + knowmeth + '\
       'usemeth + agefm + heduc + urban + electric + '\
       'bicycle + nevermarr + idlnchld_noans + heduc_noans + usemeth_noans', 
             data=data_3)

# cov_type='HC1' это тип поправки гетероскедаксичности 'HC1'
fitted_3_u = m_3.fit(cov_type='HC1')
print(fitted_3_u.summary())

                            OLS Regression Results                            
Dep. Variable:                    ceb   R-squared:                       0.644
Model:                            OLS   Adj. R-squared:                  0.643
Method:                 Least Squares   F-statistic:                     463.4
Date:                Thu, 21 Nov 2019   Prob (F-statistic):               0.00
Time:                        21:37:59   Log-Likelihood:                -7734.5
No. Observations:                4348   AIC:                         1.550e+04
Df Residuals:                    4333   BIC:                         1.559e+04
Df Model:                          14                                         
Covariance Type:                  HC1                                         
                     coef    std err          z      P>|z|      [0.025      0.975]
----------------------------------------------------------------------------------
Intercept         -1.0698      0.258     -4.

Не произошло ли значимого ухудшения модели после удаления этой группы признаков? Проверьте с помощью критерия Фишера. Чему равен его достигаемый уровень значимости? Округлите до четырёх цифр после десятичной точки.

Если достигаемый уровень значимости получился маленький, верните все удалённые признаки; если он достаточно велик, оставьте модель без религии, тв и радио.

In [210]:
print("F=%f, p=%f, k1=%f" % m_2.fit().compare_f_test(m_3.fit()))
# где compare_f_test - вызов критерия Фишера, 
# m4.fit() - бОльшая модель, m5.fit() - меньшая  

F=0.919236, p=0.467231, k1=5.000000


уровень значимости p=0.467231, это много, то есть мы можем отвергнуть гипотезу, что признаки которые мы отвергли, были действительно нужны 

Признак usemeth_noans значим по критерию Стьюдента, то есть, при его удалении модель значимо ухудшится. Но вообще-то отдельно его удалять нельзя: из-за того, что мы перекодировали пропуски в usemeth произвольно выбранным значением c_{usemeth }=-1, удалять usemeth_noans и usemeth можно только вместе.

Удалите из текущей модели usemeth_noans и usemeth. Проверьте критерием Фишера гипотезу о том, что качество модели не ухудшилось. Введите номер первой значащей цифры в достигаемом уровне значимости (например, если вы получили 5.5*10^(−8), нужно ввести 8).

Если достигаемый уровень значимости получился маленький, верните удалённые признаки; если он достаточно велик, оставьте модель без usemeth и usemeth_noans.

In [200]:
# Удалите из текущей модели usemeth_noans и usemeth. 
data_4 = data_3.copy()
data_4.drop(columns=['usemeth_noans', 'usemeth'], axis=1, inplace=True)
print data_3.shape, data_4.shape

(4348, 15) (4348, 13)


In [215]:
# Проверьте критерием Фишера гипотезу о том, что качество модели не ухудшилось.
m_4 = smf.ols('ceb ~ age + educ + idlnchld + knowmeth + '\
       'agefm + heduc + urban + electric + '\
       'bicycle + nevermarr + idlnchld_noans + heduc_noans', 
             data=data_4)


print("F=%f, p=%f, k1=%f" % m_3.fit().compare_f_test(m_4.fit()))
# где compare_f_test - вызов критерия Фишера, 
# ml_3.fit() - бОльшая модель, ml_4.fit() - меньшая  

F=92.890582, p=0.000000, k1=2.000000


уровень значимости получился маленький, нужно вернуться к изучению предыдущей модели 